In [ ]:
from hydromt_sfincs import SfincsModel
from hydromt.config import configread
from hydromt.log import setuplog
from os.path import join, basename

In [ ]:
# model and data paths
model_root = r'../../3_models/SFINCS/00_base'
data_libs = [
    r'../../1_data/1_static/static_data.yml',
    r'../../1_data/2_forcing/forcing_data.yml',    
]
logger = setuplog('update', join(model_root, "hydromt.log"), log_level=10)

In [ ]:
# build model base layers
region = {'bbox': [34.33,-20.12,34.95,-19.30]}
res = 100 # resolution [m]
opt = configread('build_sfincs.ini', abs_path=True)
kwargs = opt.pop('global',{})
mod = SfincsModel(root=model_root, data_libs=data_libs, logger=logger, **kwargs)
mod.build(region=region, res=res, opt=opt)
mod.plot_basemap()

In [ ]:
# update river bathymetry and manning roughness
new_root = r'../../3_models/SFINCS/01_riv_powlaw'
mod1 = SfincsModel(model_root, mode='r', data_libs=data_libs, logger=logger)
mod1.read()
mod1.set_root(new_root)
mod1.setup_river_bathymetry(
    river_geom_fn = 'rivers_lin2019_v1', 
    river_mask_fn = 'grwl_mask', 
    rivdph_method='powlaw',
    rivwth_method='mask',
    hc=0.27,
    hp=0.40,
    river_upa = 100,
    constrain_estuary = True,
    # plot_riv_profiles = 2
)
mod1.setup_manning_roughness(
    riv_man=0.03,
    lnd_man=0.1
)  
mod1.write()
mod1.plot_basemap(geoms=[''])

In [ ]:
# update forcing
root = r'../../3_models/SFINCS/01_riv_powlaw'
for event in ['idai', 'eloise']:
    new_root = f'{root}_{event}'
    mod1 = SfincsModel(root, mode='r', data_libs=data_libs, logger=logger)
    opt = configread(f'update_sfincs_{event}.ini', abs_path=True)
    mod1.update(model_out=new_root, opt=opt)
    mod1.write_config(rel_path=f'../{basename(root)}')
    mod1.plot_forcing()